We let $X$ be an $n_x$-dimensional vector representing the input data. There are $n_x$ features. Similarly, $Y$ is a $n_y$-dimenstional vector representing the outut. For binary classification $n_y=1$.

There are $L$ layers with $n^{[l]}$ nodes in each. The input later ($X$) is considered layer zero, while the output layer is layer $L$.
The linear part of layer $l$ is written
$$ Z^{[l]}_{i} = W^{[l]}_{ij} A^{[l-1]}_{j} + b^{[L]}_i$$

Here $Z^{[l]}_{i}$ refers to the linear part of the ith node in layer $l$. There is one $Z$ vector for each layer and it has dimension $n_l$. Here $W \sim n_l \times n_{l-1}$ and $b \sim n_l$ are parameters.

The activations of a layer $l$ are
$$ A^{[l]}_i = g^{(l)}(Z^{[l]}_i)$$
where the activation function $g$ (relu, sigmoid, tanh or whatever) is applied element-wise.

We write the cost for a single training example as
$$J = - \large{(} \small Y \log\left(A^{[L]}\right) + (1-Y) \log\left(1- A^{[L]}\right) \large{)} \small\tag{13}$$

Our goal is to compute the derivatives of $J$ wrt the parameters $W$ and $b$. We do this via repeated application of the chain rule. Our approach is as follows:

1. calculate the derivative of $J$ wrt $A^{[L]}$ and $Z^{[L]}$
2. use this to calculate the derivatives of $J$ wrt $W^{[L]}$ abd $b^{[L]}$
3. show that given the derivatives for layer $l$ we can calculate all the derivatives for layer $l-1$



$$\frac{\partial J}{\partial A^{[L]}} = - \Big( \frac{Y_i}{A^{[L]}} - \frac{(1-Y)}{(1-A^{[L]})} \Big) = \frac{ A^{[L]} - Y}{ A^{[L]} ( 1 - A^{[L]} )}
$$

$$
\begin{align}
\frac{\partial J}{\partial Z^{[L]}} &= \frac{\partial J}{\partial A^{[L]}} \frac{\partial A^{[L]}}{\partial Z^{[L]}} = \frac{\partial J}{\partial A^{[L]}} g'^{[L]}(Z^{[L]}) \\
&= \frac{\partial J}{\partial A^{[L]}} A^{[L]} ( 1 - A^{[L]} ) \\
&= A^{[L]} - Y
\end{align}
$$
(last two lines only valid for sigmoid activation in layer $L$). Once we have this we can proceed to calculate the partial derivatives wrt $W$ and $b$:
$$
\frac{\partial J}{\partial W^{[l]}_{ij}} = \frac{\partial J}{\partial Z^{[l]}_i} \frac{\partial Z^{[l]}_i}{\partial W^{[l]}_{ij}} = \frac{\partial J}{\partial Z^{[l]}_i} A^{[l-1]}_j
$$
$$
\frac{\partial J}{\partial b^{[l]}_{i}} = \frac{\partial J}{\partial Z^{[l]}_i} \frac{\partial Z^{[l]}_i}{\partial b^{[l]}_{i}} = \frac{\partial J}{\partial Z^{[l]}_i}
$$
Finally for $l<L$
$$
\begin{align}
\frac{\partial J}{\partial A^{[l]}_i} &= \frac{\partial J}{\partial Z^{[l+1]}_j} \frac{\partial Z^{[l+1]}_j}{\partial A^{[l]}_i}\\
&= \frac{\partial J}{\partial Z^{[l+1]}_j} W^{[l+1]}_{ji}
\end{align}
$$

$$
\begin{align}
\frac{\partial J}{\partial Z^{[l]}_i} &= \frac{\partial J}{\partial A^{[l]}_i} \frac{\partial A^{[l]}_i}{\partial Z^{[l]}_i}\\
&= \frac{\partial J}{\partial A^{[l]}_i} g'^{[l]}(Z^{[l]}_i)
\end{align}
$$

In [ ]:
import numpy as np
np.random.seed(0)

m = 10  # number of training examples
n_x = 2 # number of features
n_y = 1 
layer_dims = [ 5, 2, 1 ]
activation_functions = ['relu', 'relu', 'sigmoid']
#layer_dims = [ 5, 1 ]
#activation_functions = ['relu', 'sigmoid']

learning_rate = 0.1
X = np.random.rand(n_x, m)
Y = np.random.rand(n_y, m)

In [ ]:
def relu(x):
    ret = np.maximum(0,x)
    return(ret)

def sigmoid(x):
    ret = 1 + np.exp(-x)
    return( 1/ret)

def compute_cost(y_label, yhat):
    size = y_label.shape[1]
    ret = np.dot(y_label, np.log(yhat).T) + np.dot((1-y_label), np.log(1-yhat).T)
    ret = ret.squeeze()
    return( -ret/size )

def initialize_params(n_x, layer_dims):
    np.random.seed(0)
    layer_dims = [n_x] + layer_dims
    ret = {}
    for i in range(1, len(layer_dims)):
        n_h = layer_dims[i]
        W = np.random.rand(n_h, layer_dims[i-1])
        b = np.zeros( (n_h, 1) ) + 0.3
        ret['W'+str(i)] = W
        ret['b'+str(i)] = b
    return(ret)

def forward_prop(X, params):
    assert( len(params)%2 == 0 )
    L = int(len(params)/2)
    ret = {}
    cache = {}
    A_prev = X
    for i in range(1,L+1):
        W = params['W'+str(i)]
        b = params['b'+str(i)]
        Z = np.dot( W, A_prev) + b
        activation_function = activation_functions[i-1]
        if activation_function == 'relu':
            A = relu(Z)
        elif activation_function == 'sigmoid':
            A = sigmoid(Z)
        else:
            raise ValueError('unknown activation function')
        cache['Z'+str(i)] = Z
        cache['A'+str(i)] = A
        A_prev = A
    return(A, cache)

def relu_deriv(v):
    return( np.greater(v,0))

def compute_grads(AL, params, cache, X, Y):
    # X is n_x x m
    # Y is 1 x m
    cache['A0'] = X
    assert( len(params)%2 == 0 )
    L = int(len(params)/2)
    
    grads = {}
    dZ = AL - Y
    for i in reversed(range(1,L+1)):
        A_next = cache['A'+str(i-1)]
        dW = np.dot(dZ, A_next.T)/m  
        db = np.sum(dZ, axis=1, keepdims = True)/m 
        grads['dW'+str(i)] = dW
        grads['db'+str(i)] = db
        if i>1:
            W = params['W'+str(i)]
            Z_next = cache['Z'+str(i-1)]
            dZ = np.dot(W.T, dZ) * relu_deriv(Z_next)
        
    return( grads )

def update_parameters( parameters, gradients, learning_rate):
    new_params = {}
    for param_name, param_value in parameters.items():
        grad = grads['d'+param_name]
        new_params[param_name] = param_value - learning_rate * grad
        #print( "updating {} from\n {} \nto\n {}".format(param_name, param_value, new_params[param_name] ) )
    return( new_params )


In [ ]:
# initialize params
params = initialize_params(n_x, layer_dims)
num_iterations = 5000

for i in range(num_iterations):
    # forward prop
    AL, cache = forward_prop(X, params)

    # compute cost
    cost = compute_cost(Y, AL)
    #print("Cost after iteration {}: {}".format( i, cost) )
    # backward prop to calculate gradients
    grads = compute_grads(AL, params, cache, X, Y)

    params = update_parameters( params, grads, learning_rate )
print( "Final cost: {}".format(cost))
